# Flux/Zygote + CUDAのデモ

Copyright (c) 2022 Tadashi Wadayama  
Released under the MIT license  
https://opensource.org/licenses/mit-license.php

In [1]:
using LinearAlgebra
using Plots
gr()
using Random
Random.seed!(1)
using Flux
using CUDA

### CUDAが利用できるかどうか確認

In [2]:
CUDA.functional()

true

### CPU計算 vs GPU計算 

各セルを2回実行して比較をしてください。

In [19]:
A = randn(200,200)
B = randn(200,200)
@time C = A*B;

  0.000256 seconds (2 allocations: 312.578 KiB)


In [20]:
A = cu(randn(200,200))
B = cu(randn(200,200))
@time C = A*B;

  0.000095 seconds (57 allocations: 1.281 KiB)


In [3]:
f(x) = 4*x[1]^2 + x[2]^2

f (generic function with 1 method)

### GPU に送りたい行列などにcu()をつける

In [6]:
x0 = cu(randn(2))

2-element CuArray{Float32, 1}:
 0.8667514
 1.6776252

### 勾配法の実装

自動微分に加えてFluxの最適化関数(勾配法)の利用

In [7]:
opt = Descent(0.1)
x = x0
for i in 1:10
    ps = params(x)
    gs = gradient(ps) do
        f(x)
    end
    Flux.Optimise.update!(opt, ps, gs)
    println(x)
end

┌ Warning: Performing scalar operations on GPU arrays: This is very slow, consider disallowing these operations with `allowscalar(false)`
└ @ GPUArrays /home/user/.julia/packages/GPUArrays/Z5nPF/src/host/indexing.jl:64


Float32[0.17335027, 1.3421001]
Float32[0.034670055, 1.0736802]
Float32[0.0069340114, 0.8589441]
Float32[0.0013868022, 0.6871553]
Float32[0.00027736044, 0.5497242]
Float32[5.547209f-5, 0.43977937]
Float32[1.1094417f-5, 0.3518235]
Float32[2.2188833f-6, 0.2814588]
Float32[4.4377668f-7, 0.22516704]
Float32[8.875534f-8, 0.18013363]


In [8]:
x

2-element CuArray{Float32, 1}:
 8.875534f-8
 0.18013363